In [1]:
import pandas as pd
import os
from PIL import Image

from tqdm import tqdm

In [2]:
INPUT_CSV = r"data/2_21_without_image.csv"
INPUT_EXCEL = r"C:\Users\user\Downloads\Without img.xlsx"
INPUT_IMAGES = r"C:\Users\user\Downloads\images\images"
OUTPUT_IMAGES = r"OUTPUT/2_21_output"
INPUT_CSV, INPUT_IMAGES

('data/2_21_without_image.csv', 'C:\\Users\\user\\Downloads\\images\\images')

In [3]:
xl = pd.ExcelFile(INPUT_EXCEL)
data = xl.parse(xl.sheet_names[1])
data

,id,external_id,provider,name,Unnamed: 4
0,2394,e40d7d1867e24f1bb4bc81fced4af27e,bigtimegaming,Dragon Born Mobile,https://altacdn.com/bf/img/games/92939-dragon-...
1,529,b79e72b50cf211e1ba00cabb5ca54b38356131a9,blueprint,Master of Atlantis,https://altacdn.com/bf/img/games/104843-atlant...
2,2648,f32e946f5e04fd941cc7ca50ce45c9360981a780,caleta,Halloween Lotto,https://altacdn.com/bf/img/games/91031-hallowe...
3,2654,bbbb115d41cf05749814ab606acdd2a99631055f,wazdan,Sizzling Bells Mobile,https://altacdn.com/bf/img/games/92219-sizzlin...
4,2657,496f119595d1cf050790b51b435496951f296f9a,tripleprofitsgames,Dinosaur Island,https://altacdn.com/bf/img/games/90582-dinosau...
...,...,...,...,...,...
7197,4215,dbf56e972b384b70b994f1ad81aabe2e,playngoasia,Super Wheel,NaN
7198,4219,671de66a49ff4c0ea34ec6fa03f2bf08,playngoasia,Temple of Wealth,NaN
7199,4225,b02b780eb78e424aa40358e2860fa782,playngoasia,The Shimmering Woods,NaN
7200,4228,eee3a51458ca473eaaf502ff585746d0,playngoasia,Tome of Madness,NaN


In [4]:
# data.to_csv(INPUT_CSV, index=False)

In [4]:
data = pd.read_csv(INPUT_CSV)
data

,ID,Hash,Provider,Name,Other
0,2394,e40d7d1867e24f1bb4bc81fced4af27e,bigtimegaming,Dragon Born Mobile,https://altacdn.com/bf/img/games/92939-dragon-...
1,529,b79e72b50cf211e1ba00cabb5ca54b38356131a9,blueprint,Master of Atlantis,https://altacdn.com/bf/img/games/104843-atlant...
2,2648,f32e946f5e04fd941cc7ca50ce45c9360981a780,caleta,Halloween Lotto,https://altacdn.com/bf/img/games/91031-hallowe...
3,2654,bbbb115d41cf05749814ab606acdd2a99631055f,wazdan,Sizzling Bells Mobile,https://altacdn.com/bf/img/games/92219-sizzlin...
4,2657,496f119595d1cf050790b51b435496951f296f9a,tripleprofitsgames,Dinosaur Island,https://altacdn.com/bf/img/games/90582-dinosau...
...,...,...,...,...,...
7197,4215,dbf56e972b384b70b994f1ad81aabe2e,playngoasia,Super Wheel,NaN
7198,4219,671de66a49ff4c0ea34ec6fa03f2bf08,playngoasia,Temple of Wealth,NaN
7199,4225,b02b780eb78e424aa40358e2860fa782,playngoasia,The Shimmering Woods,NaN
7200,4228,eee3a51458ca473eaaf502ff585746d0,playngoasia,Tome of Madness,NaN


In [13]:
data[data["ID"] == "54940267a83140bf9d45697ddceac72a"]["Name"].values[0]

'Fairy Dust Mobile'

In [11]:
rename = lambda x: x.replace("™", "").replace("’", "").replace("´", "").replace("–", "").replace("‘", "").replace("é", "e").strip() # 3033

In [21]:
name2hash = {}
for i in range(len(data)):
    i = data.iloc[i]
    name = rename(i["Name"].lower().replace("mobile", ""))
    if name in name2hash:
        if isinstance(name2hash[name], list):
            name2hash[name].append(i["ID"])
        else:
            name2hash[name] = [name2hash[name]] + [i["ID"]]
    else:
        name2hash[name] = i["ID"].lower()

In [22]:
len(list(name2hash.keys()))

5152

In [23]:
len(data)

7202

In [24]:
naming = [rename(x[:-4].lower()) for x in os.listdir(INPUT_IMAGES)]

plus = []
minus = []

for i in naming:
    if i[-1] == 'x2':
        continue
    
    if i in name2hash:
        plus += [i]
    else:
        minus += [i]

len(plus), len(minus)

(3033, 11413)

In [25]:
for name in tqdm(os.listdir(INPUT_IMAGES)):
    i = rename(name[:-4].lower())
    if i[-1] == 'x2':
        i = i[:-2]

    if i in name2hash:
        img_path = os.path.join(INPUT_IMAGES, name)
        img = Image.open(img_path)
        w, h = img.size
        if isinstance(name2hash[i], list):
            for j in name2hash[i]:
                out_path = os.path.join(OUTPUT_IMAGES, j) + ".png"   
                img.save(out_path)
        else:
            out_path = os.path.join(OUTPUT_IMAGES, name2hash[i]) + ".png"
            img.save(out_path)
            # if h > w and os.path.exists(out_path):
            #     print("vert   = ", name)
            #     img.save(out_path)
                # continue
            # if os.path.exists(out_path):
            #     print(name)
                # continue
            # img.save(out_path)

100%|██████████| 14446/14446 [04:27<00:00, 53.95it/s] 


In [20]:
name2hash[i]

[['b35f10daefd748c7b6ac097e7ec89998', '1238addbd3c9411984cff074c4ace225'],
 '77c6a519899b467985f725eda6aa0f99']

In [33]:
OUTPUT_IMAGES_VERT = "OUTPUT/2_21_vert"

In [34]:
for name in tqdm(os.listdir(INPUT_IMAGES)):
    i = rename(name[:-4].lower())
    if i in name2hash:
        continue
    
    img_path = os.path.join(INPUT_IMAGES, name)
    img = Image.open(img_path)
    w, h = img.size
    if h > w:
        out_path = os.path.join(OUTPUT_IMAGES_VERT, rename(name))
        if os.path.exists(out_path):
            print(name)
        img.save(out_path)

 19%|█▉        | 2739/14446 [00:25<01:59, 98.15it/s] 

Casino Hold’emx2.jpg


100%|██████████| 14446/14446 [02:16<00:00, 106.00it/s]


## Вопросы
Всего оригинальных картинок:
Сматченных картинок:
Не сматченных, но есть вертикальный экземпляр

In [ ]:
Privé Lounge 1x2

## Проблемы

- в картинках нету слова Mobile, но в Excell он встречается сотни раз, матчить такие случаи?
- 5400 картинок нету в Excell
 


In [22]:
img.size

(450, 338)

In [32]:
[x for x in minus if "™" in x]

['12 coins™ grand gold edition',
 '12 coins™ grand gold editionx2',
 '12 coins™ halloween edition',
 '12 coins™',
 '12 coins™x2',
 '15 coins™',
 '15 coins™x2',
 '16 coins™ halloween edition',
 '16 coins™',
 '16 coins™x2',
 '9 coins™ easter edition',
 '9 coins™ extremely light',
 '9 coins™ grand diamond edition',
 '9 coins™ grand platinum edition',
 '9 coins™ grand platinum editionx2',
 '9 tigers™',
 '9 tigers™x2',
 'aloha spirit xtralock™',
 'aloha spirit xtralock™x2',
 'big bass - hold & spinner™',
 'burning sun™ easter edition',
 'burning sun™ extremely light',
 'burning sun™ extremely lightx2',
 'burning sun™',
 'burning sun™x2',
 'choco reels™ easter',
 'choco reels™ easterx2',
 'cube mania deluxe™',
 'cube mania deluxe™x2',
 'cube mania™',
 'cube mania™x2',
 'dragons lucky 8™',
 'dragons lucky 8™x2',
 'dwarfs fortune™ easter',
 'dwarfs fortune™ easterx2',
 'el patron™',
 'el patron™x2',
 'get the gold infinireels™',
 'good luck clusterbuster™',
 'hot slot™- 777 cash out extremely 

In [ ]:
Book Of Wolves – Full Moonx2